In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from PIL import Image
from tqdm.auto import tqdm
import os
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
from transformers import CLIPVisionModel, CLIPProcessor
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import shutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
import re
import os
import pandas as pd
import multiprocessing
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib

def download_image(image_link, savefolder):
    if(isinstance(image_link, str)):
        filename = Path(image_link).name
        image_save_path = os.path.join(savefolder, filename)
        if(not os.path.exists(image_save_path)):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
            except Exception as ex:
                print('Warning: Not able to download - {}\n{}'.format(image_link, ex))
        else:
            return
    return

def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    results = []
    download_image_partial = partial(download_image, savefolder=download_folder)
    with multiprocessing.Pool(100) as pool:
        for result in tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)):
            results.append(result)
        pool.close()
        pool.join()

In [10]:
DATA_PATH = "/content/drive/MyDrive/Amazon ML challenge/amazon ml challenge/student_resource/dataset/"

df = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))
# print(df[df["sample_id"] == 286800])
df.drop(df[df["sample_id"] == 286800].index, axis = 0, inplace = True)
# print(df[df["sample_id"] == 286800])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 143, shuffle = False)

In [11]:
image_links = df['image_link'].tolist()

download_images(image_links, "/content/test_images")

len(os.listdir("/content/test_images"))

100%|██████████| 74999/74999 [11:32<00:00, 108.34it/s]


72221

In [5]:
df

,sample_id,catalog_content,image_link
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...
...,...,...,...
74995,93616,Item Name: Good Seasons Zezty Italian Salad Dr...,https://m.media-amazon.com/images/I/51e9H27lgv...
74996,249434,"Item Name: Colombina Swirled Love Tiger Pops, ...",https://m.media-amazon.com/images/I/61IpkExmVt...
74997,162217,"Item Name: Kerns, Guava Nectar, 11.5 Fl Oz Can...",https://m.media-amazon.com/images/I/A1NMggyCLz...
74998,230487,Item Name: NY SPICE SHOP Licorice Candy - 1 Po...,https://m.media-amazon.com/images/I/81P69kEP5q...


In [12]:
model_name = "openai/clip-vit-base-patch32"
clip_vision_model = CLIPVisionModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

for param in clip_vision_model.parameters():
  param.requires_grad = False

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [16]:
torch.manual_seed(143)

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     # transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                         #  std=[0.229, 0.224, 0.225])
# ])

class amazon_image_dataset(torch.utils.data.Dataset):
  def __init__(self, X, processor):
    self.X = X
    self.processor = processor

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    IMAGE_PATH = "/content/test_images/"
    image_path = self.X.iloc[index][36:]
    image_path = os.path.join(IMAGE_PATH, image_path)

    try:
      img = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
      print(f"Missing file: {image_path}")
      return None

    processed_image = self.processor(images=img, return_tensors="pt")['pixel_values'].squeeze(0)
    return processed_image

In [18]:
torch.manual_seed(143)
batch_size = 32

X = df['image_link'].copy()

test_dataset = amazon_image_dataset(X, processor = processor)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

print(len(test_loader))

2344


In [19]:
class SMAPELoss(nn.Module):
    def __init__(self, epsilon=1e-2):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        denominator = (torch.abs(y_true) + torch.abs(y_pred)) / 2.0 + self.epsilon
        mape = torch.abs(y_pred - y_true) / denominator
        return torch.mean(mape)

class CLIP_embedding_model(nn.Module):
  def __init__(self, clip_model):
    super().__init__()
    self.clip_model = clip_model
  def forward(self, pixel_values):
    outputs = self.clip_model(pixel_values = pixel_values)
    return outputs.pooler_output

model = CLIP_embedding_model(clip_vision_model).to(device)

In [22]:
model.eval()

all_embeddings = []

with torch.no_grad():
    for images in tqdm(test_loader):
        images = images.to(device)

        embeddings = model(images)
        all_embeddings.append(embeddings.cpu().numpy())

final_embeddings = np.vstack(all_embeddings)


print(f"Successfully generated embeddings for {final_embeddings.shape[0]} images.")
print(f"Shape of the final embeddings array: {final_embeddings.shape}")

from google.colab import files

np.save("test_clip_embeddings.npy", final_embeddings)
files.download("test_clip_embeddings.npy")
print("Embeddings saved and downloaded successfully!")

 13%|█▎        | 298/2344 [42:22<4:50:58,  8.53s/it]


KeyboardInterrupt: 